# Binary case

In [ ]:
import numpy as np
from ollama_models import ollama_models
from load_saved_questions import load_saved_questions
from call_local_llm import call_local_llm
from community_forecast import community_forecast
import pandas as pd
from flatten_dict import flatten_dict
from datetime import datetime
from gather_research import gather_research
from tqdm import tqdm
import time, load_secrets
from extract_forecast import *
from error import error

load_secrets.load_secrets()

models = ollama_models()
model = models[0]
question = load_saved_questions([17763])[0]
print('START model', model, 'id', id)
start_time = time.time()
questions = [question]
id = question.id_of_question
id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}
df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
df['id_of_post'] = [q.id_of_post for q in questions]
df['id_of_question'] = [q.id_of_question for q in questions]
df['question_options'] = df['question_options'].apply(repr)
df['today'] = datetime.now().strftime("%Y-%m-%d")
df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)
df['error'] = ''
df1 = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title',
    'question_resolution_criteria', 'question_fine_print', 'question_type', 
    'question_description', 'question_options', 'question_group_variable', 'question_question_weight',
    'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
    'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']].copy()
df2, rag = gather_research(df1, True, model)
df3 = df2[['id_of_question', 'title',
        'question_resolution_criteria', 'question_type', 
         'question_description','crowd',
          'research', 'asknews', 'learning']].copy()

row = df3.iloc[0]

prompt2 = f"""
You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
{row.title}
```

```news
{row.asknews}

```research
{row.research}
```

The question defines an event and resolves as YES if the event occurs, otherwise NO. 
You are forecasting the probability of YES.
The event is described by the RESOLUTION CRITERIA.

```resolution_criteria
{row.question_resolution_criteria}
```

The last thing you write is your final answer as: "Probability: ZZ%", 0-100
"""

answer_mistral = call_local_llm(prompt2, model)
answer_mistral2 = call_local_llm(prompt2, model)
answer_mistral3 = call_local_llm(prompt2, model)

forecasts = [extract_probability_from_response_as_percentage_not_decimal(x)/100.0
             for x in [answer_mistral, answer_mistral2, answer_mistral3]]

forecast = float(np.median(forecasts))

prompt = f"""
Summarize the gist pf the rationale or thinking of the following answers from 3 different forecasters to a single problem. 

```forecast 1
{answer_mistral}
```

```forecast 2
{answer_mistral2}
```

```forecast 3
{answer_mistral3}
```

DO NOT REFER TO THE 3 FORECASTERS.  PRESENT THIS AS YOUR OWN THINKING, YOUR OWN RATIONALE.  Use as your final the median forecast which is

```median forecast
{forecast}
```
"""

rationale = call_local_llm(prompt, model)

row.forecast = rationale
row.prediction = forecast
row.error = error(row)

fdir = f'forecast_{model}'
os.makedirs(fdir, exist_ok=True)

